In [9]:
# Genetic Algorithm

import numpy as np
import random
#필요한 라이브러리를 가져온다.
#전체 이동 거리를 계산하는 함수
def calculate_total_distance(tour, distance_matrix):
    total_distance = 0 # 전체 이동거리 0으로 초기화 
    for i in range(len(tour) - 1): #입력받은 경로 리스트의 인덱스를 반복하며 , 마지막 인덱스는 제외된다. 
        total_distance += distance_matrix[tour[i]][tour[i + 1]] 
        # 현재 노드와 다음 노드간의 거리를 거리 행렬에서 가져와 total_distance에 더한다.
    total_distance += distance_matrix[tour[-1]][tour[0]]  #마지막 노드에서 시작 노드로의 거리를 추가한다. 
    return total_distance #구한 총 거리를 계산해 나타낸다.

#초기해 생성 
def generate_initial_solution(num_nodes): 
    return [num_nodes - 1] + random.sample(list(range(num_nodes - 1)), num_nodes - 1)
'''노드의 개수를 입력받아 초기해를 생성한다. 
시작 노드는 목적지로 정해지기 때문에 시작 노드를 제외한 다른 노드들의 무작위 순열로 구성된다.
시작 노드는 반드시 마지막에 추가된다.'''

# 초기 모집단 생성
def initialize_population(population_size, num_nodes):
    # 주어진 모집단 크기와 노드의 수에 따라 초기 모집단을 생성하는 함수이다. 
    population = [generate_initial_solution(num_nodes) for _ in range(population_size)]
    return population

''' generate_initial_solution 함수를 모집단 크기 만큼 호출하고,
그 결과를 통해 얻은 초기 해별들로 이루어진 리스트를 생성한다.
population 리스트 반환한다.'''

#두 부모 해 간에 교차 수행
# 교차 확률에 따라 교차를 수행하거나 수행하지 않는다.
def crossover(parent1, parent2, crossover_probability):
    if random.random() < crossover_probability:
        # 랜덤으로 생성한 수가 교차 확률보다 작을 경우-> 교차 수행
        crossover_point = random.randint(1, len(parent1) - 1)
        # 교차 지점 선택
        child = parent1[:crossover_point] + [city for city in parent2 if city not in parent1[:crossover_point]]
        # 교차 지점 이전까지는 부모1의 일부를 자식에게 복사
        #나머지 부분은 부모2에서 부모1에 이미 존재하지 않는 도시들을 찾아 자식에 추가

        return child
        # 자식 반환
    else:
        return parent1
        # 교차를 수행하지 않으면 부모1 그대로 반환

# 돌연변이 함수
def mutate(solution):
    i, j = sorted(random.sample(range(1, len(solution)), 2))
    # 무작위로 선택한 두 지점의 인덱스를 오름차순으로 정렬
    #range(1, len(solution)) -> 첫번째 노드를 선택하지 않도록 하기 위함

    mutated_solution = solution[:i] + list(reversed(solution[i:j + 1])) + solution[j + 1:]
    # 선택된 구간을 뒤집어 새로운 해법(투어) 생성

    return mutated_solution
    # 뒤집힌 부분을 포함하여 새로운 해법 반환

# 유전 알고리즘 구현 함수- 거리 행렬, 모집단 크기, 반복수, 돌연변이 발생 확률, 교차 발생 확률을 매개변수로 할당
def genetic_algorithm(distance_matrix, population_size, num_generations, mutation_probability, crossover_probability):
    num_nodes = len(distance_matrix)
    # 노드의 개수 저장
    population = initialize_population(population_size, num_nodes)
    # initialize_population 함수를 사용하여 초기 해법으로 이루어진 초기 모집단을 생성한다.
    best_solution = min(population, key=lambda x: calculate_total_distance(x, distance_matrix))
    # calculate_total_distance 함수를 사용하여 초기 모집단에서 가장 좋은 해법을 찾아
    # best_solution에 저장한다.

    for generation in range(num_generations):
        # num_generations 만큼 반복하면서 세대를 진행한다.
        parents = random.sample(population, k=2)
        # 현재 모집단에서 무작위로 두 부모를 선택한다.

        # Crossover 단계
        child = crossover(parents[0], parents[1], crossover_probability)
        # 선택된 두 부모를 사용하여 교차를 수행하고 자식을 생성한다.

        # Mutation 단계
        if random.random() < mutation_probability:
        # 랜덤으로 선택한 값이 돌연변이 확률보다 낮으면 돌연변이를 진행한다.
            child = mutate(child)

        # 적합도 평가 단계
        population.append(child)
        # 생성된 자식을 현재 모집단에 추가한다.
        population = sorted(population, key=lambda x: calculate_total_distance(x, distance_matrix))[:population_size]
        # 모집단을 거리 값에 따라 정렬한 후에 가장 좋은 해만 남기도록 모집단의 크기를 조절한다.

        # 최적해 업데이트
        current_best = min(population, key=lambda x: calculate_total_distance(x, distance_matrix))
        # 현재의 모집단에서 최적해를 설정한다.

        if calculate_total_distance(current_best, distance_matrix) < calculate_total_distance(best_solution, distance_matrix):
            best_solution = current_best
        '''현재의 모집단에서 최적해에서의 총 거리 값을 계산하고
        지금까지의 best_solution의 거리 값을 계산해 비교하여 
        더 나은 해가 나오면 best_solution을 업데이트 한다.'''

        # 각 세대(시행)에서의 최적해의 노선과 그 노선의 거리 값을 계산한다.
        print(f"Generation {generation + 1}: Best Distance = {calculate_total_distance(best_solution, distance_matrix)}")
        print(f"Best Tour: {best_solution + [best_solution[0]]}")  

    return best_solution
    # 최종적을 찾은 최적해를 반환한다.


# 파라미터 설정
population_size = 50 #모집단 크기, 각 세대에 속하는 해 개수
num_generations = 1000 # 총 세대 수, 알고리즘이 실행되는 총 반복 횟수
mutation_probability = 0.2 # 돌연변이가 발생할 확률
crossover_probability = 0.8  # 교차가 발생할 확률

#거리 행렬
distance_matrix = [
    [0, 6276, 1144, 7317, 8888, 5023, 8497, 3435, 1291],
    [6331, 0, 5658, 3904, 6799, 1995, 6408, 3429, 7145],
    [1252, 5548, 0, 6589, 8674, 4295, 8283, 2891, 1611],
    [7421, 3145, 6585, 0, 6598, 3236, 8677, 5036, 8072],
    [9262, 7115, 8589, 5907, 0, 7469, 1287, 6360, 10076],
    [5139, 1917, 4303, 2958, 7020, 0, 6629, 2267, 5790],
    [8502, 6355, 7829, 6440, 480, 6709, 0, 5600, 9316],
    [4222, 3328, 3549, 5227, 6208, 2136, 5817, 0, 5036],
    [1428, 6992, 1748, 8105, 9462, 5811, 9071, 4089, 0]
]

# 유전 알고리즘 실행
best_genetic_solution = genetic_algorithm(distance_matrix, population_size, num_generations, mutation_probability,
                                          crossover_probability)

# 모든 시행이 끝난 후의 최적해의 노선과 그때의 거리 값 출력
print("Best Genetic Tour:", best_genetic_solution + [best_genetic_solution[0]])
print("Best Genetic Distance:", calculate_total_distance(best_genetic_solution, distance_matrix))

Generation 1: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 3, 7, 5, 0, 2, 8]
Generation 2: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 3, 7, 5, 0, 2, 8]
Generation 3: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 3, 7, 5, 0, 2, 8]
Generation 4: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 3, 7, 5, 0, 2, 8]
Generation 5: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 3, 7, 5, 0, 2, 8]
Generation 6: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 3, 7, 5, 0, 2, 8]
Generation 7: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 3, 7, 5, 0, 2, 8]
Generation 8: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 3, 7, 5, 0, 2, 8]
Generation 9: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 3, 7, 5, 0, 2, 8]
Generation 10: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 3, 7, 5, 0, 2, 8]
Generation 11: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 3, 7, 5, 0, 2, 8]
Generation 12: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 3, 7, 5, 0, 2, 8]
Generation 13: Best Distance = 34853
Best Tour: [8, 1, 6, 4, 